# Study torsional angles of organic molecules

In this example we will create a number of solute-in-solvent systems and we will use unbiased molecular dynamics (MD) and metadynamics (MetaD) simulations to study the conformational space of the solute molecules.


The first step is to load the necessary libraries:

In [1]:
from sim_launch_py.classes import Project
from sim_launch_py.external import molecules as mols
from sim_launch_py.plumed import write_plumed_file as wrtplumed
import numpy as np

import nglview as nv

Now we define the solute and solvent species:

In [2]:
mols_path="/home/matteo/Work/organic_molecules_simulations"

sulfamerazine={"pdb":"{}/Structures/sulfamerazine/sulfamerazine.pdb".format(mols_path),
                          "top":"{}/Topologies/sulfamerazine.top".format(mols_path),
                          "resname":"SUR"}

toluene={"pdb":"{}/Structures/toluene/toluene.pdb".format(mols_path),
                     "top":"{}/Topologies/toluene.top".format(mols_path),
                     "density":870.,
                     "MW":92.141,
                     "resname":"TOL"}

sulfamerazine["rotatables"]=mols.findTorsionalAngles(sulfamerazine['pdb'])

[18:46:55] Enabling RDKit 2019.09.3 jupyter extensions
RDKit WARNING: [18:46:55] Enabling RDKit 2019.09.3 jupyter extensions


In our example we will study sulfamerazine in toluene

In [3]:
view=nv.show_structure_file(sulfamerazine['pdb'])
view.add_unitcell(opacity=1)
view.camera='orthographic'
view

NGLWidget()

In [4]:
view=nv.show_structure_file(toluene['pdb'])
view.camera='orthographic'
view

NGLWidget()

Create a new Project object:

In [5]:
pname='sample_metad'
p=Project.new_project(name=pname,path='./{}'.format(pname),overwrite=True)


New Project: sample_metad


Create systems and populate them

In [6]:
name='{}_{}'.format("sulfamerazine","toluene")
p.add_system(name)

# get the system object
s=p.find_system_by_name(name)

# define the size of the simulation box. Needed to insert molecules.
side=3.5 # nm, side of the simulation box
V=side*side*side*0.97 #nm**3, volume of the simulation box
s.add_box(side,shape='cubic')

# insert one molecule of solute
s.insert_molecules("sulfamerazine",sulfamerazine['pdb'], nmol=1, final_conf='{}.pdb'.format("sulfamerazine"))

# center the solute in the box (optional)
s.center_box()

view=nv.show_structure_file(s.last_saved_structure)
view.add_unitcell(opacity=1)
view.camera='orthographic'
view

**************************************************
Created system sulfamerazine_toluene
**************************************************
1 molecules of residue SUR have been added. The file Systems/sulfamerazine_toluene/sulfamerazine.pdb has been created.


NGLWidget()

In [7]:
# estimate the number of solvent molecules for the target density
nmol_solv= int(toluene["density"]/toluene["MW"]*6.022*1e23/1e24*V)

# insert the solvent molecules
s.insert_molecules("toluene",toluene['pdb'],nmol=nmol_solv, initial_conf=s.last_saved_structure, final_conf='{}.pdb'.format(name))

view=nv.show_structure_file(s.last_saved_structure)
view.add_unitcell(opacity=1)
view.camera='orthographic'
view

236 molecules of residue TOL have been added. The file Systems/sulfamerazine_toluene/sulfamerazine_toluene.pdb has been created.


NGLWidget()

In [8]:
# create the topology file of the system
s.species[sulfamerazine["resname"]]['top']=sulfamerazine['top']
s.species[toluene["resname"]]['top']=toluene['top']
s.create_topology('topol.top')

atom type ca already existing, continuing...
atom type c3 already existing, continuing...
atom type hc already existing, continuing...
atom type ha already existing, continuing...


Once the initial configuration of system has been defined, we can start to define the simulations we want to perform.
For each simulation we define a dict with the options/parameters for the simulation, then we add the simulation.

In [9]:
em_dict={'path_mdp':'/home/matteo/Work/organic_molecules_simulations/sim_launch_py/mdp/em.mdp',
         'maxwarn':2, 'nsteps':1000, 'coordinates':s.last_saved_structure}

s.add_simulation('em','em',simulation_dict=em_dict)

nvt_dict={'path_mdp': '/home/matteo/Work/organic_molecules_simulations/sim_launch_py/mdp/nvt.mdp',
          'maxwarn':2,
          'coordinates':'{}/{}.gro'.format(s.simulations[-1].path,s.simulations[-1].name),
          'posre':'{}/{}.gro'.format(s.simulations[-1].path,s.simulations[-1].name),
          'nsteps':50000}

s.add_simulation('nvt','posre',simulation_dict=nvt_dict)

npt_dict={'path_mdp': '/home/matteo/Work/organic_molecules_simulations/sim_launch_py/mdp/npt.mdp',
          'maxwarn':2,
          'coordinates':'{}/{}.gro'.format(s.simulations[-1].path,s.simulations[-1].name),
          'posre':'{}/{}.gro'.format(s.simulations[-1].path,s.simulations[-1].name),
          'nsteps':50000}

s.add_simulation('npt','posre',simulation_dict=npt_dict)
        
md_dict={'path_mdp': '/home/matteo/Work/organic_molecules_simulations/sim_launch_py/mdp/mdparrinello.mdp',
         'maxwarn':2, 'coordinates':'{}/{}.gro'.format(s.simulations[-1].path,s.simulations[-1].name),
         'nsteps':5000000, 'plumed':'plumed.dat'}
                     
s.add_simulation('md','md',simulation_dict=md_dict)

sim=s.find_simulation_by_name('md')

Simulation em(type em) will start from configuration ./Systems/sulfamerazine_toluene/sulfamerazine_toluene.pdb.
Added simulation em (0) to system sulfamerazine_toluene (0).
Simulation nvt(type posre) will start from configuration ./Systems/sulfamerazine_toluene/em/em.gro.
Added simulation nvt (1) to system sulfamerazine_toluene (0).
Simulation npt(type posre) will start from configuration ./Systems/sulfamerazine_toluene/nvt/nvt.gro.
Added simulation npt (2) to system sulfamerazine_toluene (0).
Simulation md(type md) will start from configuration ./Systems/sulfamerazine_toluene/npt/npt.gro.
Added simulation md (3) to system sulfamerazine_toluene (0).


We can define collective variables (CV) to be monitored during the simulation. 
To do that we use the Simulation.add_cv() method for each CV we want to add. 

Then we create the plumed file with the sim_launch_py.plumed.write_plumed_file() method

In [10]:
for irot,rot in enumerate(sulfamerazine['rotatables']):
    sim.add_cv('dih_{}'.format(irot),'torsion',cv_dict={'atoms':np.array(rot)+1})

wrtplumed('{}/plumed.dat'.format(sim.path),sim,colvar='COLVAR')
        
myriad_dict={'wallclock':'12:00:00',
             'job_name':s.name,
             'mpi':12,
             'omp':6,
             'gmx_bin':'gmx_mpi'}

s.create_run_script('run.myriad',platform='myriad',platform_dict=myriad_dict)

Written run script file in ./Systems/sulfamerazine_toluene/run.myriad


Finally we can save the project and run the simulations on our HPC system of choice.

In [11]:
p.save()        



Saving Project...done
